# Selection Diagrams

This notebook is for creating diagrams of how selection works, based on *real* data from the yt sample datasets.

We'll construct and highlight diagrams showing selection of:

 * Slices
 * Regions
 * Spheres
 
This should be straightforward since we can use the built-in yt selection routines.

In [1]:
import drawSvg as draw
import yt
import numpy as np
import ipywidgets

In [2]:
yt.utilities.logger.disable_stream_logging()

In [3]:
#ds = yt.load("~/data/IsolatedGalaxy/galaxy0030/galaxy0030")
#ds = yt.load("/home/matthewturk/data/EnzoKelvinHelmholtz/DD0011/DD0011")
#ds = yt.load("/home/matthewturk/data/PopIII_mini/DD0034/DD0034")
ds = yt.load("~/data/enzo_tiny_cosmology/DD0046/DD0046")

In [4]:
s = ds.sphere("c", 0.5)

Parsing Hierarchy : 100%|██████████| 211/211 [00:00<00:00, 6182.83it/s]


In [5]:
xax = 0
yax = 1
margin = 20
width = height = 500

In [6]:
def make_image_data(source):
    selected_grids = source.selector.select_grids(
        source.ds.index.grid_left_edge,
        source.ds.index.grid_right_edge,
        source.ds.index.grid_levels
    )
    frb = source.integrate("density", axis="z").to_frb(1.0, (512, 512))
    frb_data = np.log10(frb["density"][::-1,:].v)
    frb_bitmap = yt.write_image(frb_data, None)
    frb_bitmap[frb_data == -np.inf,3] = 0
    image_data = yt.write_bitmap(frb_bitmap, None)
    
    im = draw.Image(data = image_data, x = margin, y = margin, width = width - 2*margin,
                height = height - 2*margin, mimeType="image/png", opacity = 1)
    return im, selected_grids

In [7]:
def make_svg(source, selected = 'black', unselected = 'grey', show_image = True):
    c = draw.Drawing(width = width, height = height)

    im, sg = make_image_data(source)

    if show_image:
        c.append(im)

    scale_x = lambda v: margin + (width - 2*margin) * v
    scale_y = lambda v: margin + (height - 2*margin) * v
    domain = draw.Rectangle(scale_x(0), scale_y(0),
                            scale_x(1.0) - scale_x(0.0),
                            scale_y(1.0) - scale_y(0.0),
                            fill = 'none', stroke_width=2.0,
                            stroke = 'black')
    c.append(domain)
    def draw_grid(stroke, stroke_width, LE, RE):
        grid_outline = draw.Rectangle(
            scale_x(LE[xax]),
            scale_y(LE[yax]),
            scale_x(RE[xax]) - scale_x(LE[xax]),
            scale_y(RE[yax]) - scale_y(LE[yax]),
            fill = '#AAAAAA', fill_opacity=0.0, stroke = stroke, stroke_width=stroke_width)
        return grid_outline
    [c.append(draw_grid(unselected, 0.5, grid.LeftEdge.v, grid.RightEdge.v)) for grid in ds.index.grids[~sg]]
    [c.append(draw_grid(selected, 1.0, grid.LeftEdge.v, grid.RightEdge.v)) for grid in ds.index.grids[sg]]
    return c

In [8]:
cv = (0.0, 1.0, 0.01)

In [9]:
@ipywidgets.interact(left_edge_x = cv, left_edge_y = cv, left_edge_z = cv,
                     right_edge_x = cv, right_edge_y = cv, right_edge_z = cv)
def make_reg(left_edge_x = 0.0, left_edge_y = 0.0, left_edge_z = 0.0,
             right_edge_x = 1.0, right_edge_y = 1.0, right_edge_z = 1.0):
    r = ds.r[left_edge_x:right_edge_x, left_edge_y:right_edge_y, left_edge_z:right_edge_z]
    return make_svg(r, 'black', 'grey')

interactive(children=(FloatSlider(value=0.0, description='left_edge_x', max=1.0, step=0.01), FloatSlider(value…

In [10]:
@ipywidgets.interact(center_x = cv, center_y = cv, center_z = cv, radius = cv,
                    show_image = True)
def make_sphere(center_x = 0.5, center_y = 0.5, center_z = 0.5,
             radius = 0.15, show_image = True):
    c = make_svg(ds.sphere((center_x, center_y, center_z), radius), 'black', 'grey', show_image = show_image)
    return c

interactive(children=(FloatSlider(value=0.5, description='center_x', max=1.0, step=0.01), FloatSlider(value=0.…

In [11]:
sp1 = ds.sphere("c", 0.20)
sp1_svg = make_svg(sp1)
sp1_svg.saveSvg("../content/images/selectors/sp1.svg")

sp2 = ds.sphere((0.7, 0.7, 0.5), 0.25)
sp2_svg = make_svg(sp2)
sp2_svg.saveSvg("../content/images/selectors/sp2.svg")

dobj1 = sp1 + sp2
dobj1_svg = make_svg(dobj1)
dobj1_svg.saveSvg("../content/images/selectors/sp1_and_sp2.svg")

reg1 = ds.r[0.2:0.3,0.2:0.3,:]
reg1_svg = make_svg(reg1)
reg1_svg.saveSvg("../content/images/selectors/reg1.svg")

reg2 = ds.r[0.4:0.95, 0.4:0.95, 0.4:0.6]
reg2_svg = make_svg(reg2)
reg2_svg.saveSvg("../content/images/selectors/reg2.svg")

dobj2 = reg2 - sp2
dobj2_svg = make_svg(dobj2)
dobj2_svg.saveSvg("../content/images/selectors/reg2_not_sp2.svg")

/home/matthewturk/conda-py3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log10
  
